# Overview

PyTrack is designed as an object oriented mapper for [DVC](https://dvc.org/).
For an introduction we highly recommend reading the [DVC Getting Started](https://dvc.org/doc/start).
Besides version controlled data management, DVC provides method for building a dependency graph, tracking parameters, comparing metrics, reducing computational overhead and querying multiple runs.

**Why does it need an object-oriented mapper?**

Whilst DVC provides all this functionality it is designed to be programming language independent.
This can require writing custom python scripts, reading and writing config files and managing depencencies.

PyTrack is designed to make these steps as easy and well integrated with Python as possible.
In comparison  to the DVC backbone, it is aimed directly at python developeres and therefore allows a highly adapted and optimized interface.

## Structure
PyTrack is based on two parts, a class decorator and descriptors for e.g. parameter tracking.

### PyTrack class decorator
        
The PyTrack decorator converts a Python class into a DVC stage, by wrapping the `__init__`, `__call__` and `run`. It handles almost all of the required steps to create a DVC stage.

### PyTrackOptions

PyTrackOptions are custom Descriptors (implementing a custom `__get__` and `__set__` method) that are used to track parameters as well as defining dependencies, metrics or other outputs.

In [1]:
!git init
!dvc init

rm: refusing to remove '.' or '..' directory: skipping '.'
rm: refusing to remove '.' or '..' directory: skipping '..'
Initialized empty Git repository in /tikhome/fzills/PycharmProjects/py-track/docs/source/Overview/01_Intro/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


## Stages
DVC organizes its pipeline in multiple stages.
When using PyTrack we can write our code inside a jupyter notebook.
We can make use of this functionality by setting the `nb_name` config as follows:

In [2]:
from pytrack import PyTrack, config
config.nb_name = "01_Intro.ipynb"

To define a stage we can start with a Python class implementing only a `run` method.
The class must implement a `run` method, which is the entry point for the computation executed by DVC.
To convert the class into a PyTrack stage we apply the `PyTrack()` decorator to it.

There are two things we can do with the stage at this point.
1. Create a new instance of the stage:
After instantiation of the new instance we can set attributes, e.g. parameters or dependencies.
Ideally now expensive calculations are required at this stage, because we only want to create a frame for our method.
2. Calling the stage:
If no explizit `__call__` method is defined, PyTrack will add one to our stage.
This method is usually the place to interface with the user, passing parameters, dependencies, outputs, etc. .
After the call, the class will write the stage to the `dvc.yaml` file and we are ready to run the stage via DVC.

In [5]:
@PyTrack()
class Stage0:
    def run(self):
        pass
    
stage_0 = Stage0()
stage_0()

2021-10-19 09:00:44,925 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/py-track.


[NbConvertApp] Converting notebook 01_Intro.ipynb to script


2021-10-19 09:00:48,410 (WARNING): --- Writing new DVC file! ---
2021-10-19 09:00:48,410 (WARNING): Overwriting existing configuration!


[NbConvertApp] Writing 10241 bytes to 01_Intro.py


2021-10-19 09:00:51,883 (INFO): Creating 'dvc.yaml'
Adding stage 'Stage0' in 'dvc.yaml'

To track the changes with git, run:

	git add dvc.yaml outs/.gitignore



In [6]:
!tree

.
├── 01_Intro.ipynb
├── config
│   └── pytrack.json
├── dvc.yaml
├── outs
└── src
    └── Stage0.py

3 directories, 4 files


We can see, that PyTrack has created a `dvc.yaml` file for us (using DVC in the backend).

In [8]:
from IPython.display import Pretty, display
display(Pretty("dvc.yaml"))

stages:
  Stage0:
    cmd: python3 -c "from src.Stage0 import Stage0; Stage0(load=True).run()"
    deps:
    - src/Stage0.py
    outs:
    - outs/Stage0.json


We can see, that it does run `Stage0(load=True).run()` I.e. this method must be able to run on its own.
In addition to that a config file has been created, but our stage does not yet depend on it.
This makes sense because no parameters have been defined.
The file `outs/Stage0.json` will be used to store PyTrack results and is required for dependency handling.

In [9]:
display(Pretty("config/pytrack.json"))

{
    "Stage0": {},
    "default": null
}

We can now use `dvc  repro` to execute our code, which in result does nothing yet

In [10]:
!dvc repro

Running stage 'Stage0':                                               core>
> python3 -c "from src.Stage0 import Stage0; Stage0(load=True).run()"
2021-10-19 09:21:40,190 (WARNING): No results found!
Generating lock file 'dvc.lock'                                                 
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


### PyTrack Results
We can see, that the stage is run without issues.
Unfortunately, the stage we just created doesn't do anything.
In our first example we would like to create a random number in our stage and save it.
We can do this utilizing `DVC.result` which is a special type of a DVC outs file, managed by PyTrack.
We do this by defining a class level attribute.
This is similar to setting a Python `@property` where `__get__` and `__set__` has some custom handling assigend to it.
In comparison the the `@property` we do not need to think about the `getter/setter`.

In [12]:
from pytrack import DVC
from random import randrange

@PyTrack()
class RandomNumer:
    number = DVC.result()
    
    def run(self):
        self.number = randrange(10)

random_number = RandomNumer()
random_number()

2021-10-19 09:24:11,845 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/py-track.


[NbConvertApp] Converting notebook 01_Intro.ipynb to script


2021-10-19 09:24:15,200 (WARNING): --- Writing new DVC file! ---
2021-10-19 09:24:15,201 (WARNING): Overwriting existing configuration!


[NbConvertApp] Writing 10225 bytes to 01_Intro.py


2021-10-19 09:24:18,324 (INFO): Adding stage 'RandomNumer' in 'dvc.yaml'

To track the changes with git, run:

	git add outs/.gitignore dvc.yaml



We can access the outcome of our stage by passing `load=True`. Doing this on our stage will give us a warning and simply return None.
This is, because we haven't actually executed the `run` method yet. Again, this is done via `dvc repro`

In [7]:
RandomNumer(load=True).number

2021-10-15 15:37:04,460 (WARNING): No results found!


In [15]:
!dvc repro

Stage 'Stage0' didn't change, skipping                                core>
Running stage 'MaxRandomNumer':
> python3 -c "from src.MaxRandomNumer import MaxRandomNumer; MaxRandomNumer(load=True).run()"
2021-10-19 09:24:33,879 (WARNING): No results found!
Updating lock file 'dvc.lock'                                                   

Running stage 'RandomNumer':
> python3 -c "from src.RandomNumer import RandomNumer; RandomNumer(load=True).run()"
2021-10-19 09:24:35,467 (WARNING): No results found!
Updating lock file 'dvc.lock'                                                   

Running stage 'WriteToFile':
> python3 -c "from src.WriteToFile import WriteToFile; WriteToFile(load=True).run()"
2021-10-19 09:24:37,081 (WARNING): No results found!
Updating lock file 'dvc.lock'                                                   

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


Now we can have a look at our result and work with it.

In [9]:
RandomNumer(load=True).number

6

Because we are using DVC, rerunnig the stage via `dvc repro` will not result in a new computation, but instead it will use the cached value.
Changing this is explained later.

### PyTrack arguments
Currently our stage will always yield a random number in the hard coded range 0-9.
PyTrack stages become increasingly more interesting when introducing custom parameters. We can now start by adding a maximum value to our stage.


In [13]:
@PyTrack()
class MaxRandomNumer:
    number = DVC.result()
    maximum = DVC.params()
    
    def __call__(self, maximum):
        self.maximum = maximum
    
    def run(self):
        self.number = randrange(self.maximum)

max_random_number = MaxRandomNumer()
max_random_number(maximum=512)

2021-10-19 09:24:18,337 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/py-track.


[NbConvertApp] Converting notebook 01_Intro.ipynb to script


2021-10-19 09:24:21,268 (WARNING): --- Writing new DVC file! ---
2021-10-19 09:24:21,269 (WARNING): Overwriting existing configuration!


[NbConvertApp] Writing 10225 bytes to 01_Intro.py


2021-10-19 09:24:24,286 (INFO): Adding stage 'MaxRandomNumer' in 'dvc.yaml'

To track the changes with git, run:

	git add dvc.yaml outs/.gitignore



In [11]:
!dvc repro

Running stage 'MaxRandomNumer':                                       core>
> python3 -c "from src.MaxRandomNumer import MaxRandomNumer; MaxRandomNumer(load=True).run()"
2021-10-15 15:37:15,560 (WARNING): No results found!
Updating lock file 'dvc.lock'                                                   

Stage 'RandomNumer' didn't change, skipping
Stage 'Stage0' didn't change, skipping

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


In [12]:
MaxRandomNumer(load=True).number

448

### Custom Types and Files

When using arguments PyTrack can handle the most basic python types and also some more complex ones like `pathlib.Path`.
In the following example we introduce using paths as arguments and writing data to a custom output file.
Therefore we use `DVC.outs`

In [14]:
from pathlib import Path

@PyTrack()
class WriteToFile:
    filename: Path = DVC.outs()
    
    def __call__(self, filename: Path):
        self.filename = filename
    
    def run(self):
        self.filename.write_text('Lorem Ipsum')
        
    def read_from_file(self):
        print(self.filename.read_text())

write_to_file = WriteToFile()
write_to_file(filename=Path("outs", "example.txt"))

2021-10-19 09:24:24,299 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/py-track.


[NbConvertApp] Converting notebook 01_Intro.ipynb to script


2021-10-19 09:24:27,613 (WARNING): --- Writing new DVC file! ---
2021-10-19 09:24:27,614 (WARNING): Overwriting existing configuration!


[NbConvertApp] Writing 10225 bytes to 01_Intro.py


2021-10-19 09:24:30,782 (INFO): Adding stage 'WriteToFile' in 'dvc.yaml'

To track the changes with git, run:

	git add dvc.yaml outs/.gitignore



In [16]:
!dvc repro

Stage 'WriteToFile' didn't change, skipping                           core>
Stage 'MaxRandomNumer' didn't change, skipping
Stage 'Stage0' didn't change, skipping
Stage 'RandomNumer' didn't change, skipping
Data and pipelines are up to date.


We can see, that a file in `outs` with our filename has been created. The file can be generated anywhere inside the DVC repository, but using the already exising `outs` directory can be handy.
We can again load the stage and have a look at the filename and read from it.

In [15]:
WriteToFile(load=True).filename

PosixPath('outs/example.txt')

In [16]:
WriteToFile(load=True).read_from_file()

Lorem Ipsum


At this point our `dvc.yaml` file has grown a bit and looks like the following

In [17]:
display(Pretty("dvc.yaml"))

stages:
  Stage0:
    cmd: python3 -c "from src.Stage0 import Stage0; Stage0(load=True).run()"
    deps:
    - src/Stage0.py
    outs:
    - outs/Stage0.json
  RandomNumer:
    cmd: python3 -c "from src.RandomNumer import RandomNumer; RandomNumer(load=True).run()"
    deps:
    - src/RandomNumer.py
    outs:
    - outs/RandomNumer.json
  MaxRandomNumer:
    cmd: python3 -c "from src.MaxRandomNumer import MaxRandomNumer; MaxRandomNumer(load=True).run()"
    deps:
    - src/MaxRandomNumer.py
    params:
    - config/pytrack.json:
      - MaxRandomNumer.params
    outs:
    - outs/MaxRandomNumer.json
  WriteToFile:
    cmd: python3 -c "from src.WriteToFile import WriteToFile; WriteToFile(load=True).run()"
    deps:
    - src/WriteToFile.py
    outs:
    - outs/WriteToFile.json
    - outs/example.txt


We can also look at our `pytrack.json` file investigating the passed arguments:

In [18]:
display(Pretty("config/pytrack.json"))

{
    "Stage0": {},
    "default": null,
    "RandomNumer": {},
    "MaxRandomNumer": {
        "params": {
            "maximum": 512
        }
    },
    "WriteToFile": {
        "outs": {
            "filename": {
                "Path": "outs/example.txt"
            }
        }
    }
}

### PyTrack Init

As you may have already noticed we have not created an `__init__` yet. Arguments are passed to the `__call__` and `PyTrackOptions` (DVC.<...>) are defined on a class level. The following example will illustrate, why using the `__init__` can lead to confusing results.
Therefore we need to keep in mind, that DVC runs the follwing command: 

    python3 -c "from src.Stage0 import Stage0; Stage0(load=True).run()"
    
which we will use to imitate `dvc repro` in the following.

In [17]:
@PyTrack()
class InitStage:
    def __init__(self, value = "Not defined"):
        self.value = value
        
    def run(self):
        print(self.value)

2021-10-15 15:37:25,706 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/py-track.


[NbConvertApp] Converting notebook 01_Intro.ipynb to script
[NbConvertApp] Writing 8652 bytes to 01_Intro.py


In [18]:
init_stage = InitStage(value='Lorem Ipsum')
init_stage()
print(init_stage.value)

2021-10-15 15:37:28,839 (WARNING): --- Writing new DVC file! ---
2021-10-15 15:37:28,840 (WARNING): Overwriting existing configuration!
2021-10-15 15:37:31,662 (INFO): Adding stage 'InitStage' in 'dvc.yaml'

To track the changes with git, run:

	git add dvc.yaml outs/.gitignore

Lorem Ipsum


In [19]:
InitStage(load=True).run()

2021-10-15 15:37:31,679 (WARNING): No results found!
Not defined


We can see, that our passed value is not available during the command that is executed by `DVC`. This is important to keep in mind, when using PyTrack. The issue can be easily solved by using `DVC.params()`. Altough possible, it should be avoided to define them within the `__init__` and go for class level definitions. Nevertheless, the `__init__` can be used for e.g., defining class attributes or setting `PyTrackOption`.
We can therefore extend our `MaxRandomNumber` in the following way by a constand minimum value:

In [20]:
@PyTrack()
class InitMaxRandomNumer:
    number = DVC.result()
    maximum = DVC.params()
    
    def __init__(self):
        self.minimum = 0
    
    def __call__(self, maximum):
        self.maximum = maximum
        
    def run(self):
        self.number = randrange(self.minimum, self.maximum)

init_max_random_number = InitMaxRandomNumer()
init_max_random_number(maximum=512)

2021-10-15 15:37:31,688 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/py-track.


[NbConvertApp] Converting notebook 01_Intro.ipynb to script


2021-10-15 15:37:34,699 (WARNING): --- Writing new DVC file! ---
2021-10-15 15:37:34,700 (WARNING): Overwriting existing configuration!


[NbConvertApp] Writing 8652 bytes to 01_Intro.py


2021-10-15 15:37:37,769 (INFO): Adding stage 'InitMaxRandomNumer' in 'dvc.yaml'

To track the changes with git, run:

	git add dvc.yaml outs/.gitignore



In [21]:
!dvc repro

Running stage 'InitStage':                                            core>
> python3 -c "from src.InitStage import InitStage; InitStage(load=True).run()"
2021-10-15 15:37:39,957 (WARNING): No results found!
Not defined
Updating lock file 'dvc.lock'                                                   

Stage 'Stage0' didn't change, skipping
Stage 'MaxRandomNumer' didn't change, skipping
Stage 'RandomNumer' didn't change, skipping
Stage 'WriteToFile' didn't change, skipping
Running stage 'InitMaxRandomNumer':
> python3 -c "from src.InitMaxRandomNumer import InitMaxRandomNumer; InitMaxRandomNumer(load=True).run()"
2021-10-15 15:37:41,006 (WARNING): No results found!
Updating lock file 'dvc.lock'                                                   

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


In [22]:
InitMaxRandomNumer(load=True).number

208

Because this is a essential property of PyTrack and differs from most other Python code the following example DOES NOT work, because dvc will try to run `InitMaxRandomNumer(load=True).run()` without passing a value to `maximum` and therefore resulting in an error!

```python

@PyTrack()
class InitMaxRandomNumerWrong:
    number = DVC.result()
    maximum = DVC.params()
    
    def __init__(self, maximum):
        self.minimum = 0
        self.maximum = maximum
        
    def run(self):
        self.number = randrange(self.minimum, self.maximum)
```

what does work would be the follwing version. But for code clearity it should be avoided if possible and the `__call__` should be utilized.
Sometimes a combined approach might be inevitable, because e.g., upon class instatiation a generated value shall be passed and later a user value.

In [23]:
@PyTrack()
class InitMaxRandomNumerCheaty:
    number = DVC.result()
    maximum = DVC.params()
    
    def __init__(self, maximum = None):
        self.minimum = 0
        if maximum is not None:
            self.maximum = maximum
        
    def run(self):
        self.number = randrange(self.minimum, self.maximum)

2021-10-15 15:37:41,603 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/py-track.


[NbConvertApp] Converting notebook 01_Intro.ipynb to script
[NbConvertApp] Writing 8652 bytes to 01_Intro.py


In [24]:
init_max_random_number_cheaty = InitMaxRandomNumerCheaty(maximum=4096)
init_max_random_number_cheaty()
!dvc repro

2021-10-15 15:37:44,539 (WARNING): --- Writing new DVC file! ---
2021-10-15 15:37:44,539 (WARNING): Overwriting existing configuration!
2021-10-15 15:37:47,448 (INFO): Adding stage 'InitMaxRandomNumerCheaty' in 'dvc.yaml'

To track the changes with git, run:

	git add outs/.gitignore dvc.yaml

Stage 'Stage0' didn't change, skipping                                core>
Running stage 'InitMaxRandomNumerCheaty':
> python3 -c "from src.InitMaxRandomNumerCheaty import InitMaxRandomNumerCheaty; InitMaxRandomNumerCheaty(load=True).run()"
2021-10-15 15:37:49,754 (WARNING): No results found!
Updating lock file 'dvc.lock'                                                   

Stage 'InitStage' didn't change, skipping
Stage 'RandomNumer' didn't change, skipping
Stage 'InitMaxRandomNumer' didn't change, skipping
Stage 'WriteToFile' didn't change, skipping
Stage 'MaxRandomNumer' didn't change, skipping

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remot

In [25]:
InitMaxRandomNumerCheaty(load=True).number

723